In [1]:
from constants import REFINED_MITRE_PROMPT_TEMPLATE
result = REFINED_MITRE_PROMPT_TEMPLATE.format(
    advisory_content="Sample advisory content for testing",
    techniques_to_analyze="sample techniques to analyze")
print(result)


You are a cybersecurity expert analyzing an ICS (Industrial Control Systems) security advisory.
Your task is to map this advisory to the most relevant MITRE ATT&CK techniques.

IMPORTANT: You have been provided with pre-filtered candidate techniques that are most likely relevant.
Focus your analysis on these candidates, but you may also suggest if none are truly appropriate.

Advisory Content:
Sample advisory content for testing

Candidate MITRE ATT&CK Techniques (pre-filtered for relevance):
sample techniques to analyze

Instructions:
1. Analyze the advisory for attack vectors, vulnerabilities, and potential impacts
2. Map to 1-3 most relevant techniques from the candidates provided

Only respond with a valid JSON object in this exact format and nothig else
{
  "type": "object",
  "properties": {
    "mapped_techniques": {
      "type": "array",
      "description": "List of MITRE ATT&CK technique IDs that apply the MOST to the advisory. This could be zero, one or more techniques but

In [27]:
from constants import ENHANCED_RAG_TEMPLATE


In [28]:
from agent import ThreatIntelligenceAgent
from indexmanager import IndexManager

index_manager = IndexManager("ai-cti")
index = index_manager.load_existing_index()
agent = ThreatIntelligenceAgent(index, index_manager)


Loading MITRE ATT&CK techniques from file...
Loading existing index...


In [14]:
agent_setup = agent.setup_agent()


In [15]:
agent.chat("what is the most recent advisory")

KeyError: 'context_str'

In [29]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.prompts import PromptTemplate
from llama_index.core.response_synthesizers import ResponseMode
from constants import llm_model



qa_prompt = PromptTemplate(ENHANCED_RAG_TEMPLATE)

query_engine = index.as_query_engine(
    llm_model= llm_model, similarity_top_k=5, text_qa_template=qa_prompt
)        
querytool=  QueryEngineTool.from_defaults(
    query_engine= query_engine,
    name="advisory_search",
    description="Search and query ICS security advisories from CISA. "
                    "Use this to find information about specific vulnerabilities, "
                    "security issues, MITRE ATT&CK mappings, or threat intelligence."
)

In [30]:
from llama_index.core.agent import ReActAgent

tools = [
    querytool
    # advisory_fetch_tool,
    # mitre_mapping_tool
]

# Create ReAct agent
re_act_agent = ReActAgent.from_tools(
    tools=tools,
    llm=llm_model,
    verbose=True,
    max_iterations=10
)

In [ ]:
re_act_answer = re_act_agent.chat("which vulnerability has the highest severity. ensure your final response contains the CVE ID links")

print(re_act_answer.response)

> Running step 5edb6705-b6cb-4516-b01c-63b29a273095. Step input: which vulnerability has the highest severity
Thought: The current language of the user is English. I need to use a tool to help me answer the question about the vulnerability with the highest severity.
Action: advisory_search
Action Input: {'input': 'highest severity vulnerability'}
Observation: **Summary**:  
The advisories detail several vulnerabilities affecting various systems, with the highest severity being CVE-2023-28531, which has a CVSS v3 base score of 9.8. This vulnerability involves missing encryption of sensitive data in OpenSSH, potentially allowing unauthorized access to smartcard keys.

**Technical Details**:  
1. **CVE-2023-28531**: This vulnerability affects OpenSSH versions before 9.3, where ssh-add adds smartcard keys to ssh-agent without intended per-hop destination constraints. This could lead to unauthorized access to sensitive data.
2. **CVE-2024-6387**: A race condition in OpenSSH's server (sshd) 